In [ ]:
# Install necessary packages
!pip install pandas numpy scikit-learn keras tensorflow gradio


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import gradio as gr


In [ ]:
from google.colab import files

# Upload file
uploaded = files.upload()

# Load dataset
df = pd.read_csv(list(uploaded.keys())[0])
df['Date'] = pd.to_datetime(df['Date'])
df.head()


Saving synthetic_transaction_data.csv to synthetic_transaction_data.csv


,Date,Transaction_Amount,Location,Anomaly
0,2024-05-06,16142.70,Chennai,1
1,2024-05-14,3432.41,Chennai,0
2,2024-07-05,3480.29,Chennai,0
3,2024-01-24,2634.92,Chennai,0
4,2024-05-06,3073.52,Chennai,0


In [ ]:
# Encode location
le = LabelEncoder()
df['Location'] = le.fit_transform(df['Location'])

# Standardize numerical features
scaler = StandardScaler()
df[['Transaction_Amount', 'Location']] = scaler.fit_transform(df[['Transaction_Amount', 'Location']])


In [ ]:
# Prepare data for LSTM
seq_length = 10
X = []
for i in range(seq_length, len(df)):
    X.append(df[['Transaction_Amount', 'Location']].iloc[i-seq_length:i].values)
X = np.array(X)

# LSTM Autoencoder model
model = keras.Sequential([
    layers.LSTM(32, input_shape=(seq_length, 2), return_sequences=True),
    layers.LSTM(16, return_sequences=False),
    layers.RepeatVector(seq_length),
    layers.LSTM(16, return_sequences=True),
    layers.LSTM(32, return_sequences=True),
    layers.TimeDistributed(layers.Dense(2))
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, X, epochs=10, batch_size=32)

# Calculate reconstruction error threshold
reconstructions = model.predict(X)
mse = np.mean(np.abs(reconstructions - X), axis=(1, 2))
thresh = np.percentile(mse, 95)  # 95th percentile as anomaly threshold


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
156/156 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - loss: 0.9212
Epoch 2/10
156/156 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.8300
Epoch 3/10
156/156 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.7299
Epoch 4/10
156/156 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.6125
Epoch 5/10
156/156 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.5381
Epoch 6/10
156/156 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.4567
Epoch 7/10
156/156 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 0.3960
Epoch 8/10
156/156 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: 0.3780
Epoch 9/10
156/156 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.3318
Epoch 10/10
156/156 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.3092
156/156 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step


In [ ]:
# Isolation Forest model
iso_forest = IsolationForest(contamination=0.05, random_state=42)
iso_forest.fit(df[['Transaction_Amount', 'Location']])


IsolationForest(contamination=0.05, random_state=42)

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Email credentials and recipient
EMAIL_ADDRESS = "kamrudhinn7@gmail.com"         # Sender's email
EMAIL_PASSWORD = "gzeyvbjufrewtipt"             # App password or regular password
TO_EMAIL = "kamrudhinn.ece2023@citchennai.net"  # Recipient's email

# Email server details
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

def send_email(subject, message):
    """Function to send an email with a given subject and message."""
    try:
        # Create the message
        msg = MIMEMultipart()
        msg["From"] = EMAIL_ADDRESS
        msg["To"] = TO_EMAIL
        msg["Subject"] = subject
        msg.attach(MIMEText(message, "plain"))

        # Connect to the SMTP server and send email
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT, timeout=10) as server:
            server.starttls()                              # Secure connection
            server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)    # Login
            server.sendmail(EMAIL_ADDRESS, TO_EMAIL, msg.as_string())  # Send email

        print("✅ Email sent successfully!")

    except Exception as e:
        print(f"❌ Error: {e}")


def detect_transaction(date, transaction_amount, location):
    """Function to predict fraud and send email if detected."""
    prediction = "✅ Transaction Approved" if float(transaction_amount) < 300 else "⚠️ Suspicious Transaction Detected!"

    # Send email only if fraud is detected
    if prediction == "⚠️ Suspicious Transaction Detected!":
        send_email(
            subject="🚨 Fraud Alert: Suspicious Transaction Detected!",
            message=f"⚠️ Suspicious transaction detected on your account!\n\n"
                    f"📅 Date: {date}\n"
                    f"💸 Amount: {transaction_amount}\n"
                    f"📍 Location: {location}\n\n"
                    f"Please review your account immediately!"
        )

    return prediction


# ✅ Example Usage (Test Case)
print(detect_transaction("2024-02-22", 500, "Chennai"))  # This will trigger an email
print(detect_transaction("2024-02-22", 200, "Mumbai"))   # This will NOT trigger an email


✅ Email sent successfully!
⚠️ Suspicious Transaction Detected!
✅ Transaction Approved


In [ ]:
def predict_fraud(date, transaction_amount, location, user_email=None):
    date = pd.to_datetime(date)

    try:
        location_encoded = le.transform([location])[0]
    except ValueError:
        return f"❌ Error: Unknown location '{location}'."

    # Standardize input
    transaction_scaled = scaler.transform([[transaction_amount, location_encoded]])[0]

    # LSTM prediction
    sequence = np.array([transaction_scaled] * seq_length).reshape(1, seq_length, 2)
    reconstruction = model.predict(sequence)
    loss = np.mean(np.abs(reconstruction - sequence))
    anomaly_lstm = 1 if loss > thresh else 0

    # Isolation Forest prediction
    anomaly_if = iso_forest.predict([transaction_scaled])[0]
    anomaly_if = 1 if anomaly_if == -1 else 0

    # Final decision
    if anomaly_if == 1 or anomaly_lstm == 1:
        alert_message = f"⚠️ Fraud Detected! Transaction on {date} at {location}."
        if user_email:
            send_email(user_email, "🚨 Fraud Alert", alert_message)
        return alert_message

    return f"✅ Transaction Approved on {date} at {location}."


In [ ]:
with gr.Blocks(css="""
    .gradio-container { font-family: 'Arial'; background: #000; color: #4285F4; padding: 40px; }
    .title { font-size: 2.5em; text-align: center; color: #4285F4; margin-bottom: 30px; }
    .textbox, .output-text { border: none; background: #2C2C2C; color: #4285F4; font-size: 16px; padding: 12px; border-radius: 8px; }
    .btn-primary { background-color: #4285F4; color: #FFF; border: none; font-size: 18px; padding: 12px 20px; border-radius: 8px; cursor: pointer; width: 100%; }
    .btn-primary:hover { background-color: #3071E5; }
""") as app:

    gr.Markdown("<div class='title'> TRAP - Fraud Detection System</div>")

    with gr.Row():
        with gr.Column():
            date_input = gr.Textbox(label="📅 Date (YYYY-MM-DD)", elem_classes=["textbox"])
            amount_input = gr.Number(label="💸 Transaction Amount", elem_classes=["textbox"])
            location_input = gr.Textbox(label="📍 Location", elem_classes=["textbox"])
            email_input = gr.Textbox(label="📧 User Email (Optional)", elem_classes=["textbox"])

    submit_button = gr.Button("🚨 Detect Fraud", elem_classes=["btn-primary"])
    output_text = gr.Textbox(label="🔍 Result", elem_classes=["output-text"])

    submit_button.click(predict_fraud, inputs=[date_input, amount_input, location_input, email_input], outputs=output_text)

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c06c50ab64212631fe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
